# Regression

In [5]:
import pandas as pd
import numpy as np
import os
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.3f' % x)



In [4]:
df = pd.read_csv('../data/2008_small.zip',index_col = 0)
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1000742,2008,2,12,2,1937.000,1940,2220.000,2120,MQ,4745,...,16.000,78.000,0,NaN,0,0.000,0.000,60.000,0.000,0.000
3014085,2008,6,8,7,1802.000,1745,1924.000,1910,WN,450,...,7.000,9.000,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4122484,2008,7,23,3,900.000,900,947.000,955,AA,1241,...,7.000,11.000,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3855808,2008,7,31,4,1030.000,1034,1244.000,1246,UA,751,...,8.000,27.000,0,NaN,0,NaN,NaN,NaN,NaN,NaN
5565463,2008,10,27,1,1618.000,1623,1839.000,1848,OO,5962,...,5.000,12.000,0,NaN,0,NaN,NaN,NaN,NaN,NaN


### Construiremos una regresión simple con dos variables que claramente están relacionadas

In [ ]:
df = df.dropna(subset = ["ArrDelay"]) # Elimiar NAs basándose en columnas de interés
df = df.sample(n = 100000)
df = df.drop(columns = ["Year", "Cancelled", "Diverted"])
Y = df["ArrDelay"]
X = df[["DepDelay"]]

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.corr()

In [ ]:
df.columns

In [ ]:
df.head().style.background_gradient(cmap='coolwarm')

In [ ]:
df.corr().style.background_gradient(cmap='coolwarm')

>Ajustamos el modelo general, que no requiere parámetros

In [ ]:
regr = linear_model.LinearRegression() # Creas el modelo vacío
regr.fit(X, Y) # Aprende de los datos
Y_pred = regr.predict(X) # Usamos el modelo para generar predicciones 

In [ ]:
Y

In [ ]:
Y_pred

>Visualizamos los datos, a la vez que mostramos la recta de regresión (predicciones)

In [ ]:
plt.scatter(X[0:10000],Y[0:10000],color = "black")
plt.plot(X[0:10000].values,Y_pred[0:10000],color="Blue")
plt.show()

In [ ]:
sns.lmplot(x="DepDelay", y="ArrDelay", data = df, height=6)

>Mostramos los coeficientes del modelo, y la métrica $R^2$, que indica un porcentaje de relación

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print("Pendiente: ",regr.coef_) # Cuánto crece Y, por cada unidad de X
print("Intercept", regr.intercept_) #Punto de corte
print("R cuadrado: ",r2_score(Y, Y_pred)) # % de variabilidad de Y, explicado por X  (Bondad de ajuste)


# Y = Intercept + Pendiente * X

>El 86 % de la **variabilidad** del retraso de llegada, viene explicado por el retraso de salida
> - Retraso de llegada = 1.018 * retraso de salida - 1.91 min

> Ejem:retraso salida: `10`\
retraso llegada: `1.018 * 10 - 1.91`

>\
Flojo < 0.3\
Moderado < 0.7\
Intenso >= 0.7

In [ ]:
df[["ArrDelay", "DepDelay"]].corr().loc["ArrDelay","DepDelay"] ** 2

In [ ]:
import seaborn as sns
dfCorr = df.corr()
filteredDf = dfCorr[((dfCorr >= .5) | (dfCorr <= -.5)) & (dfCorr !=1.000)]
filteredDf = filteredDf.dropna(how="all")
filteredDf = filteredDf.T.dropna(how="all")
plt.figure(figsize=(30,10))
sns.heatmap(filteredDf, annot=True, cmap="Reds")
plt.show()

## Ejercicios

1. Ajusta varios modelos lineales simples entre pares de variables que consideres relacionadas, escribiendo la ecuación correspondiente y mostrando el gráfico

2. Haz una evaluación de los modelos

3. (Difícil!!) Usando bucle(s), cruza todas las variables numéricas entre ellas, y muestra cuáles son los modelos más potentes

4. Representa gráficamente el error del modelo, cómo lo haríamos?

### Regresión Múltiple

El modelo de regresión realmente potente es el que considera muchas variables simultaneamente, no solamente una

In [ ]:
df[["DepDelay","AirTime", "Distance", "TaxiIn", "TaxiOut"]].corr()

In [ ]:
Y = df["ArrDelay"]
X = df[["DepDelay","AirTime","TaxiIn","TaxiOut"]]

regr = linear_model.LinearRegression()
regr.fit(X, Y) 
Y_pred = regr.predict(X) 

print("Pendiente: ",regr.coef_) #Cuanto se incrementa Y, por cada incremento de X, MANTENIENDO LAS DEMÁS VARIABLES CONSTANTES
print("Intercept", regr.intercept_) 
print("R cuadrado: ",r2_score(Y, Y_pred)) 

In [ ]:
X = df[['AirTime','Distance','TaxiIn','TaxiOut']] # eliminamos DepDelay...tiene corr alto

df["DayOfWeek"] = df["DayOfWeek"].apply(str)

dummies = pd.get_dummies(data=df[['DayOfWeek']]) # Podemos añadirle Origin y Dest
X = pd.concat([X, dummies],axis=1)

In [ ]:
X

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print("Coeficientes: ",regr.coef_) # Cuanto crece Y, por cada unidad de X, manteniendo el resto de variables constantes
print("Intercept", regr.intercept_)
Y_pred = regr.predict(X)
print("R cuadrado: ",r2_score(Y, Y_pred))

In [ ]:
X = df[['AirTime','Distance','TaxiIn','TaxiOut']] 

df["Month"] = df["Month"].apply(str)
df["DayofMonth"] = df["DayofMonth"].apply(str)
df["DayOfWeek"] = df["DayOfWeek"].apply(str)

dummies = pd.get_dummies(data=df[['Month', 'DayofMonth', 'DayOfWeek']]) # Podemos añadirle Origin y Dest
X = dummies.add(X,fill_value=0)

In [ ]:
X

In [ ]:
X.columns

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print("Coeficientes: ",regr.coef_) # Cuanto crece Y, por cada unidad de X, manteniendo el resto de variables constantes
print("Intercept", regr.intercept_)
Y_pred = regr.predict(X)
print("R cuadrado: ",r2_score(Y, Y_pred))

>¿Qué pasa cuando añadimos la variable más explicativa...?

In [ ]:
X = X.add(df[["DepDelay"]],fill_value=0)
X.columns

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

# print("Coeficientes: ",regr.coef_)
Y_pred = regr.predict(X)
print("R cuadrado: ",r2_score(Y, Y_pred))

In [ ]:
print("Intercept", regr.intercept_)

In [ ]:
print("Coeficientes: ",regr.coef_)

In [ ]:
Y_pred

In [ ]:
Y

### Ejercicios

1. Ajusta el modelo de regresión lineal simple con los datos de iris, intentando predecir la longitud del petalo

2. Mejora el modelo anterior usando todas las variables, incluida la especie

In [ ]:
sns.lmplot(data=df,x = "sepal_length" ,y = "petal_length")

In [ ]:
sns.lmplot(data=df,x = "sepal_length" ,y = "petal_length", hue = "species")

In [ ]:
sns.lmplot(data=df[df.species == "setosa"],x = "sepal_length" ,y = "petal_length")

3. Ajusta un modelo de regresión (múltiple o simple) para cada una de las especies. Cuál funciona mejor?!
Te atreves a hacerlo con un bucle?

(EXTRA) 3.1 Convierte este código a función para usarlo con cualquier dataframe

4. El modelo de regresión lineal está fuertemente relacionado con la correlación entre columnas. Crea una matriz de correlaciones que muestre qué columnas tienen más relación entre ellas

# Validación interna y externa de los modelos

#### Diferencia principal
- Con la validación interna *hacemos trampas*, ya que evaluamos el modelo con los mismos datos con el que lo hemos entrenado.
- Con la validación externa, usamos unos datos para entrenar el modelo, y otros distintos para evaluarlo. Es mucho mejor, ya que el modelo "no conoce" los datos con los que va a ser evaluado

In [ ]:
df = pd.read_csv('../data/2008.zip')
df = df.dropna(subset = ["ArrDelay"])
df = df.sample(frac=1).head(100000)

In [ ]:
X = df[['AirTime','Distance','TaxiIn','TaxiOut']]  # Añadirle el DepDelay es hacer un poco de trampa, ya que hay mucha correlación...
# dummies = pd.get_dummies(data=df[['Month', 'DayofMonth', 'DayOfWeek']])
# X = dummies.add(X,fill_value=0)

Y = df['ArrDelay']

>Creación del conjunto training y el conjunto test con `train_test_split`

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1)

>Evaluación interna

In [ ]:
modelo = linear_model.LinearRegression()
modelo.fit(X_train, y_train)
prediccionesINTERNA = modelo.predict(X_train)
print("R2: ",r2_score(y_train, prediccionesINTERNA))

>Evaluación externa

In [ ]:
prediccionesEXTERNA = modelo.predict(X_test)
print("R2: ",r2_score(y_test, prediccionesEXTERNA))

### Enriquecer los modelos

Juntamos algunas de las nuevas variables que hemos aprendido a generar

In [15]:
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

# Columnas básicas
df = pd.read_csv("../data/2008_small.zip",nrows = 5e3)
df = df.dropna(subset = ["ArrDelay"])
df = df[df.AirTime > 0]

df = df.sample(frac=1)

X = df[['AirTime','Distance','TaxiIn','TaxiOut']]

X.head()

,AirTime,Distance,TaxiIn,TaxiOut
2191,44.000,261,4.000,11.000
4944,107.000,852,12.000,17.000
549,218.000,1972,12.000,10.000
4454,91.000,447,5.000,16.000
1298,43.000,231,13.000,18.000


In [16]:
# Cálculo de polinomios

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

X = pd.DataFrame(X_poly, columns = poly.get_feature_names(input_features=X.columns))

X.head()

,1,AirTime,Distance,TaxiIn,TaxiOut,AirTime^2,AirTime Distance,AirTime TaxiIn,AirTime TaxiOut,Distance^2,Distance TaxiIn,Distance TaxiOut,TaxiIn^2,TaxiIn TaxiOut,TaxiOut^2
0,1.000,44.000,261.000,4.000,11.000,1936.000,11484.000,176.000,484.000,68121.000,1044.000,2871.000,16.000,44.000,121.000
1,1.000,107.000,852.000,12.000,17.000,11449.000,91164.000,1284.000,1819.000,725904.000,10224.000,14484.000,144.000,204.000,289.000
2,1.000,218.000,1972.000,12.000,10.000,47524.000,429896.000,2616.000,2180.000,3888784.000,23664.000,19720.000,144.000,120.000,100.000
3,1.000,91.000,447.000,5.000,16.000,8281.000,40677.000,455.000,1456.000,199809.000,2235.000,7152.000,25.000,80.000,256.000
4,1.000,43.000,231.000,13.000,18.000,1849.000,9933.000,559.000,774.000,53361.000,3003.000,4158.000,169.000,234.000,324.000


In [17]:
# Generar Columnas de interés

X["VelocidadMedia"] = X["Distance"] / X["AirTime"] * 60
X["logAirTime"] = np.log(X["AirTime"])
df["Weekends"] = False
df["Weekends"][df.DayOfWeek.isin([6,7])] = True
# df["Ruta"] = df["Origin"] + "-" + df["Dest"]

X.head()

,1,AirTime,Distance,TaxiIn,TaxiOut,AirTime^2,AirTime Distance,AirTime TaxiIn,AirTime TaxiOut,Distance^2,Distance TaxiIn,Distance TaxiOut,TaxiIn^2,TaxiIn TaxiOut,TaxiOut^2,VelocidadMedia,logAirTime
0,1.000,44.000,261.000,4.000,11.000,1936.000,11484.000,176.000,484.000,68121.000,1044.000,2871.000,16.000,44.000,121.000,355.909,3.784
1,1.000,107.000,852.000,12.000,17.000,11449.000,91164.000,1284.000,1819.000,725904.000,10224.000,14484.000,144.000,204.000,289.000,477.757,4.673
2,1.000,218.000,1972.000,12.000,10.000,47524.000,429896.000,2616.000,2180.000,3888784.000,23664.000,19720.000,144.000,120.000,100.000,542.752,5.384
3,1.000,91.000,447.000,5.000,16.000,8281.000,40677.000,455.000,1456.000,199809.000,2235.000,7152.000,25.000,80.000,256.000,294.725,4.511
4,1.000,43.000,231.000,13.000,18.000,1849.000,9933.000,559.000,774.000,53361.000,3003.000,4158.000,169.000,234.000,324.000,322.326,3.761


In [18]:
# Añadir columnas dummies

X["Month"] = df["Month"].apply(str).reset_index(drop = True)
X["DayofMonth"] = df["DayofMonth"].apply(str).reset_index(drop = True)
X["DayOfWeek"] = df["DayOfWeek"].apply(str).reset_index(drop = True)
X["Origin"] = df["Origin"]
X["Dest"] = df["Dest"]
X["UniqueCarrier"] = df['UniqueCarrier']
# X["Ruta"] = df["Ruta"]

dummies = pd.get_dummies(data=X[['Month', 'DayofMonth', 'DayOfWeek',"Origin","Dest",'UniqueCarrier']])


X = X.drop(columns = ['Month','DayOfWeek',"DayofMonth","Origin","1","Dest",'UniqueCarrier']) # No podemos mantener columnas tipo string

X.head()

,AirTime,Distance,TaxiIn,TaxiOut,AirTime^2,AirTime Distance,AirTime TaxiIn,AirTime TaxiOut,Distance^2,Distance TaxiIn,Distance TaxiOut,TaxiIn^2,TaxiIn TaxiOut,TaxiOut^2,VelocidadMedia,logAirTime
0,44.000,261.000,4.000,11.000,1936.000,11484.000,176.000,484.000,68121.000,1044.000,2871.000,16.000,44.000,121.000,355.909,3.784
1,107.000,852.000,12.000,17.000,11449.000,91164.000,1284.000,1819.000,725904.000,10224.000,14484.000,144.000,204.000,289.000,477.757,4.673
2,218.000,1972.000,12.000,10.000,47524.000,429896.000,2616.000,2180.000,3888784.000,23664.000,19720.000,144.000,120.000,100.000,542.752,5.384
3,91.000,447.000,5.000,16.000,8281.000,40677.000,455.000,1456.000,199809.000,2235.000,7152.000,25.000,80.000,256.000,294.725,4.511
4,43.000,231.000,13.000,18.000,1849.000,9933.000,559.000,774.000,53361.000,3003.000,4158.000,169.000,234.000,324.000,322.326,3.761


A veces volver a juntar dataframes puede dar algunos problemas, así que podemos solucionarlo con

In [19]:
for col in dummies.columns:
    X[col] = dummies[col]

In [20]:
X.head()

,AirTime,Distance,TaxiIn,TaxiOut,AirTime^2,AirTime Distance,AirTime TaxiIn,AirTime TaxiOut,Distance^2,Distance TaxiIn,...,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,44.000,261.000,4.000,11.000,1936.000,11484.000,176.000,484.000,68121.000,1044.000,...,0,1,0,0,0,0,0,0,0,0
1,107.000,852.000,12.000,17.000,11449.000,91164.000,1284.000,1819.000,725904.000,10224.000,...,0,0,0,0,0,0,0,1,0,0
2,218.000,1972.000,12.000,10.000,47524.000,429896.000,2616.000,2180.000,3888784.000,23664.000,...,0,0,0,0,0,0,0,0,0,0
3,91.000,447.000,5.000,16.000,8281.000,40677.000,455.000,1456.000,199809.000,2235.000,...,0,0,0,0,0,1,0,0,0,0
4,43.000,231.000,13.000,18.000,1849.000,9933.000,559.000,774.000,53361.000,3003.000,...,0,0,0,0,1,0,0,0,0,0


#### Ajuste Final del modelo

Ajustamos el modelo de regresión ampliado

In [22]:
from sklearn.metrics import mean_squared_error, r2_score

Y = df['ArrDelay']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=2)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

prediccionesINTERNA = regr.predict(X_train)
print("R2 INTERNA: ",r2_score(y_train, prediccionesINTERNA))

prediccionesEXTERNA = regr.predict(X_test)
print("R2 EXTERNA: ",r2_score(y_test, prediccionesEXTERNA))

R2 INTERNA:  0.23853159858947304
R2 EXTERNA:  0.0014707402960632


### Ejercicio final

Intenta aplicar algunos de los apartados que hemos visto en este ejemplo sobre los datos iris (o otro dataset de tu elección). Es especialmente importante la parte de validación interna y externa.

Puedes añadir alguna variable adicional, o algún tipo de preprocesamiento al modelo

Representa gráficamente los resultados

¡Explora algunas opciones adicionales!

https://seaborn.pydata.org/tutorial/regression.html

© Netmind S.L.

Todos los derechos reservados. Este documento (v1.00) ha sido diseñado para el uso exclusivo del cliente que atiende a esta formación.

Ninguna parte de este documento puede ser reproducida, distribuida o transmitida en cualquier forma o por cualquier medio sin el permiso previo por escrito de Netmind.